In [1]:
import os

os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/kuldeepchamoli889/datascipro.mlflow"

os.environ["MLFLOW_TRACKING_USERNAME"] = "kuldeepchamoli889"

os.environ["MLFLOW_TRACKING_PASSWORD"] = "f7dc259dffe49264b9f21aaf89b516e5a7be1730"


In [2]:
import os
%pwd

'/home/kuldeep-chamoli/Desktop/Mtech/krish_naik/MLOPS/datascipro/research'

In [3]:
os.chdir("../")
%pwd

'/home/kuldeep-chamoli/Desktop/Mtech/krish_naik/MLOPS/datascipro'

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str
    

In [8]:
from src.datascipro.constants import *
from src.datascipro.utils.common import read_yaml, create_directories, save_json

In [9]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH):
        self.config=read_yaml(Path(config_filepath))
        self.params=read_yaml(Path(params_filepath))
        self.schema=read_yaml(Path(schema_filepath))

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config=self.config.model_evaluation
        params=self.params.ElasticNet
        schema=self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config=ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            target_column = schema.name,
            mlflow_uri="https://dagshub.com/kuldeepchamoli889/datascipro.mlflow"
        )
        return model_evaluation_config

In [11]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib


In [15]:
import mlflow
import mlflow.sklearn
import pandas as pd
import numpy as np
from urllib.parse import urlparse
from pathlib import Path
import joblib
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from src.datascipro.utils.common import save_json

class ModelEvaluation:
    def __init__(self, config):
        self.config = config

    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2

    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[self.config.target_column]

        mlflow.set_tracking_uri(self.config.mlflow_uri)
        tracking_uri_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            predicted = model.predict(test_x)
            rmse, mae, r2 = self.eval_metrics(test_y, predicted)

            # Save metrics locally
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("mae", mae)
            mlflow.log_metric("r2", r2)

            # Register model only if not using local file store
            if tracking_uri_type_store != "file":
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel")
            else:
                mlflow.sklearn.log_model(model, "model")


In [16]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation=ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2025-04-10 19:20:42,527:INFO: common: YAML file: /home/kuldeep-chamoli/Desktop/Mtech/krish_naik/MLOPS/datascipro/config/config.yaml loaded successfully]
[2025-04-10 19:20:42,529:INFO: common: YAML file: params.yaml loaded successfully]
[2025-04-10 19:20:42,533:INFO: common: YAML file: schema.yaml loaded successfully]
[2025-04-10 19:20:42,535:INFO: common: Created directory at: artifacts]
[2025-04-10 19:20:42,536:INFO: common: Created directory at: artifacts/model_evaluation]


[2025-04-10 19:20:43,009:INFO: common: JSON file saved at: artifacts/model_evaluation/metrics.json]


2025/04/10 19:20:56 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'ElasticnetModel'.
2025/04/10 19:21:03 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ElasticnetModel, version 1
Created version '1' of model 'ElasticnetModel'.


🏃 View run thoughtful-crow-935 at: https://dagshub.com/kuldeepchamoli889/datascipro.mlflow/#/experiments/0/runs/803117ecec1045cbbba8ee4e8f785a5f
🧪 View experiment at: https://dagshub.com/kuldeepchamoli889/datascipro.mlflow/#/experiments/0
